# Deep Learning Basics for Imaging

---
## Learning Objectives
By the end of this module, learners will be able to:
- Understand what a Convolutional Neural Network (CNN) is and why it's useful in image analysis.
- Recognize the importance of deep learning in image segmentation tasks.
- Identify key Python libraries used for deep learning in imaging.
- Prepare input and interpret output for image-based deep learning models.
- Implement and evaluate a simple CNN-based segmentation pipeline.

---
## Convolutional Neural Network
A Convolutional Neural Network (CNN) is a type of artificial neural network designed specifically to process and learn from images.

**Key building blocks:**
- **Convolution layers:** detect features like edges, textures, shapes.
- **Pooling layers:** reduce image size while retaining important features.
- **Fully connected layers:** combine features to make predictions.

CNNs learn by adjusting filters (kernels) during training using large labeled datasets.

**Analogy:** Think of CNN as a set of digital microscopes that learn to detect patterns (e.g., nuclei, membranes) automatically.

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Conv2D(16, (3, 3), activation='relu', input_shape=(128, 128, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(2, activation='softmax')  # example: background vs cell
])

model.summary()

### Exercise
- Modify the CNN above to have one more convolutional layer.
- Change the activation function in the dense layer to 'sigmoid' and observe how it affects the output.

---
## Deep Learning for Segmentation
Traditional methods (thresholding, watershed) struggle with:
- Noise
- Irregular shapes
- Overlapping cells

**Deep learning segmentation learns directly from annotated examples:**
- Learns what “cell” or “background” looks like.
- Adapts to variations in shape, size, and texture.
- **Common models:** U-Net, Cellpose, StarDist

### Hands-On
Visual comparison of classical vs deep learning:

In [ ]:
import skimage.io
from skimage.filters import threshold_otsu
import matplotlib.pyplot as plt

image = skimage.io.imread('example_image.tif')
thresh = threshold_otsu(image)
binary = image > thresh

plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.title("Original")
plt.imshow(image, cmap='gray')
plt.subplot(1,2,2)
plt.title("Threshold Segmentation")
plt.imshow(binary, cmap='gray')
plt.show()

Then show how deep learning (e.g. using Cellpose) improves it:

In [ ]:
from cellpose import models
model = models.Cellpose(model_type='cyto')
masks, flows, styles, diams = model.eval(image, channels=[0,0])

plt.imshow(masks, cmap='jet')
plt.title("Cellpose Segmentation")
plt.show()

### Exercise
- Apply both thresholding and Cellpose to your own image.
- Compare the segmentation results.

---
## Popular Libraries: `TensorFlow`, `PyTorch`, `Keras`

| Library        | Use Case          | Notes                                    |
| -------------- | ----------------- | ---------------------------------------- |
| **TensorFlow** | Industrial scale  | Google-developed, popular for production |
| **PyTorch**    | Research          | Easy debugging, very Pythonic            |
| **Keras**      | Beginner-friendly | Now part of TensorFlow                   |


For beginners, Keras (via TensorFlow) is ideal due to its high-level API and clear syntax.

### Hands-On

In [ ]:
# TensorFlow + Keras Example
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential([
    Dense(32, activation='relu', input_shape=(100,)),
    Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
# PyTorch Example
import torch.nn as nn

class SimpleNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(100, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

model = SimpleNet()

### Exercise
- Modify the above models to change number of neurons or activation.
- Try loading an image dataset and pass one image through the model.

---
## Model Input/Output for Imaging
- **Input:** Images as NumPy arrays, shape often (height, width, channels)
- **For grayscale:** channels = 1
- **For RGB:** channels = 3

**Output depends on task:**
- **Classification:** One label per image
- **Segmentation:** One label per pixel (output mask of same size)

**Segmentation Workflow:**
- **Preprocess:** resize, normalize
- **Model:** CNN or U-Net
- **Output:** binary or multiclass mask

### Hands-On

In [ ]:
import numpy as np
from skimage.transform import resize

img = skimage.io.imread('cell_image.tif')
img_resized = resize(img, (128,128), preserve_range=True).astype(np.float32)
img_input = img_resized[np.newaxis, ..., np.newaxis]  # Shape: (1, 128, 128, 1)
python
Copy
Edit
# Dummy prediction
pred_mask = model.predict(img_input)
plt.imshow(pred_mask[0,...,0], cmap='gray')
plt.title("Predicted Mask")
plt.show()

### Exercise
- Take your own image and reshape it for input to a CNN.
- Visualize the predicted output mask.

---
## Mini Project: Training a Simple U-Net on Synthetic Data
**Goal:** Train a U-Net on synthetic blobs to learn segmentation.

**Tasks:**

**Load data:**

**Define a small U-Net (simplified):**

In [ ]:
def simple_unet(input_shape):
    inputs = tf.keras.Input(shape=input_shape)
    x = layers.Conv2D(16, 3, activation='relu', padding='same')(inputs)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(32, 3, activation='relu', padding='same')(x)
    x = layers.UpSampling2D()(x)
    outputs = layers.Conv2D(1, 1, activation='sigmoid')(x)
    return tf.keras.Model(inputs, outputs)

model = simple_unet((128, 128, 1))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

**Train and evaluate:**

In [ ]:
model.fit(images[..., np.newaxis], masks[..., np.newaxis], epochs=5, batch_size=8)

**Predict and visualize:**

In [ ]:
preds = model.predict(images[:5, ..., np.newaxis])
plt.imshow(preds[0,...,0], cmap='gray')

---
## Module Summary
| Topic                          | Concept                                | Hands-On                             | Outcome                                  |
| ------------------------------ | -------------------------------------- | ------------------------------------ | ---------------------------------------- |
| What is a CNN?                 | Understand convolutional layers        | Build a simple CNN                   | Learned how image features are extracted |
| Deep Learning for Segmentation | Importance over classical methods      | Cellpose vs thresholding             | Saw quality improvements with DL         |
| Libraries                      | Overview of TensorFlow, PyTorch, Keras | Defined models in both               | Understood basic syntax differences      |
| Input/Output                   | Shape, format of image data            | Preprocess image and predict         | Able to run predictions on custom images |
| Mini Project                   | Train U-Net on synthetic shapes        | Data generation, training, inference | Full pipeline for segmentation           |